In [1]:
######################################     Modules     #######################################
import sys
from time import process_time
import os
import pickle

sys.path.append('../.')
from verbose_module import routing_instances as verb
sys.path.append('../../../.')
import pIRPgym


# computer_name = input("Running experiment on mac? [Y/n]")
# if computer_name == '': 
path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPgym/'
experiments_path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Classic Instances/'
# else: 
#     path = 'C:/Users/jm.betancourt/Documents/Research/pIRPgym/'
#     experiments_path = 'G:/Mi unidad/Research/Supply Chain Analytics/Experiments/Classic Instances/'


instances = dict()
instances['Li'] = [i for i in os.listdir(path+'/pIRPgym/Instances/CVRP Instances/dCVRP/Li') if i[-3:]=='vrp']
instances['Golden'] = [i for i in os.listdir(path+'/pIRPgym/Instances/CVRP Instances/dCVRP/Golden') if i[-3:]=='vrp']
instances['Uchoa'] = [i for i in os.listdir(path+'pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
instances['Li'].sort();instances['Golden'].sort()
instances['Uchoa'].sort();instances['Uchoa'] = instances['Uchoa'][1:]+[instances['Uchoa'][0]]

Policies = ['NN','RCL','GA']

verbose = True

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


# dCVRP Instances

In [2]:
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)

for inst_set in ['Li','Golden']:
    if verbose: verb.print_head(Policies,inst_set,show_gap=True)
    
    for instance in instances[inst_set]:
        purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)
        if verbose: string = verb.print_inst(inst_set,instance,inst_gen.M,benchmark[1],benchmark[0])
        
        for policy in Policies:
            end=False
            if policy==Policies[-1]: end=True
            # print(f'{inst_set}/{policy}/{instance[:-4]}.pkl')
            with open(experiments_path+f'{inst_set}/{policy}/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
            
            if policy in ['NN','GA']:
                if verbose: string = verb.print_routing_update(string,performance[1],len(performance[0]),
                                                                                 performance[3],True,benchmark,end=end)
            elif policy=='RCL':
                if verbose: string = verb.print_routing_update(string,performance[0],performance[1],
                                                                                 performance[2],True,benchmark,intervals=performance[3],end=end)
    
    print('\n')

********************************************************  Li set Instances  ********************************************************
--------|-----------------------|	  NN 	 	|	  	  	 	RCL 	 	 	|	  GA 	 	|
Inst	|   M 	  Veh	 Obj	| t(s)	 #Veh 	 gap 	| t(s)	 #Veh 	 mean 	median	 stdev	 min	 max	| t(s)	 #Veh 	 gap 	|
--------------------------------------------------------------------------------------------------------------------------------------------
Li 21 	|  560	  10 	16213 	| 0.03	  13	 28.0	| 0.16	  14.8	 46.63	 35.18 	27.38	  24.5	 130.31	| 3.05	  13	 28.0	|
Li 22 	|  600	  15 	14499 	| 0.00	  22	 31.72	| 0.09	  24.3	 47.54	 42.62 	11.58	  36.62	 74.5	| 3.20	  22	 31.72	|
Li 23 	|  640	  10 	18801 	| 0.00	  14	 41.85	| 0.13	  13.8	 43.66	 32.81 	30.65	  20.93	 151.15	| 3.08	  14	 41.85	|
Li 24 	|  720	  10 	21389 	| 0.06	  13	 30.59	| 0.16	  14.8	 49.19	 35.24 	34.23	  26.75	 168.69	| 3.30	  13	 30.59	|
Li 25 	|  760	  19 	16666 	| 0.05	  28	 31.56	| 0.16	  30.1	 40.96	 39.63 	7

### GA 

In [27]:
POPULATION_SIZES = [250,750,1000,2500]
ELITE_PROPORTIONS = [0.05,0.15,0.3]
MUTATION_RATES = [0.25,0.5,0.75]

combinations = [(p,e,m) for p in POPULATION_SIZES for e in ELITE_PROPORTIONS for m in MUTATION_RATES]
per = {com:list() for com in combinations}

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)


for instance in instances['Li'][0:8]:
    purchase,benchmark = inst_gen.upload_CVRP_instance('Li',instance)

    # for (p,e,m) in combinations:
    with open(experiments_path+f'Li/GA/{instance[:-4]}_{p}_{e}_{m}.pkl', 'rb') as file:
        # Use pickle.dump to serialize and save the dictionary to the file
        performance = pickle.load(file)

    per[(p,e,m)].append(round((performance[1]-benchmark[0])/benchmark[0],2))
        # print(f'{p}_{e}_{m} \t',round((performance[1]-benchmark[0])/benchmark[0],2))
        # print(f'{p}_{e}_{m} \t',performance[1])

per = {com:sum(per[com])/len(per[com]) for com in combinations}

FileNotFoundError: [Errno 2] No such file or directory: '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Classic Instances/Li/GA/Li_21_250_0.05_0.25.pkl'

In [23]:
for com in combinations:
    print(com, '\t', per[com])

(250, 0.05, 0.25) 	 0.7749999999999999
(250, 0.05, 0.5) 	 0.78125
(250, 0.05, 0.75) 	 0.7737499999999999
(250, 0.15, 0.25) 	 0.77
(250, 0.15, 0.5) 	 0.78
(250, 0.15, 0.75) 	 0.7775000000000001
(250, 0.3, 0.25) 	 0.76875
(250, 0.3, 0.5) 	 0.7875000000000001
(250, 0.3, 0.75) 	 0.7712500000000002
(750, 0.05, 0.25) 	 0.7675000000000001
(750, 0.05, 0.5) 	 0.76375
(750, 0.05, 0.75) 	 0.7575
(750, 0.15, 0.25) 	 0.7662499999999999
(750, 0.15, 0.5) 	 0.7562500000000001
(750, 0.15, 0.75) 	 0.77
(750, 0.3, 0.25) 	 0.7575
(750, 0.3, 0.5) 	 0.7725000000000001
(750, 0.3, 0.75) 	 0.7675000000000001
(1000, 0.05, 0.25) 	 0.76625
(1000, 0.05, 0.5) 	 0.74625
(1000, 0.05, 0.75) 	 0.7425
(1000, 0.15, 0.25) 	 0.73875
(1000, 0.15, 0.5) 	 0.7525
(1000, 0.15, 0.75) 	 0.925
(1000, 0.3, 0.25) 	 0.755
(1000, 0.3, 0.5) 	 0.7675000000000001
(1000, 0.3, 0.75) 	 0.76875
(2500, 0.05, 0.25) 	 0.75125
(2500, 0.05, 0.5) 	 0.7374999999999999
(2500, 0.05, 0.75) 	 0.7474999999999999
(2500, 0.15, 0.25) 	 0.7575000000000001
(

# CVRP Instances

In [7]:
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_head(Policies,inst_set,show_gap=True)

for instance in instances[inst_set]:
    purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)
    if verbose: string = verb.print_inst(inst_set,instance,inst_gen.M,benchmark[1],benchmark[0])
    
    for policy in Policies:
        end=False
        if policy==Policies[-1]: end=True

        with open(experiments_path+f'{inst_set}/{policy}/{instance[:-4]}.pkl', 'rb') as file:
            # Use pickle.dump to serialize and save the dictionary to the file
            performance = pickle.load(file)
        
        if policy=='NN':
            if verbose: string = verb.print_routing_update(string,performance[1],len(performance[0]),
                                                                                performance[3],True,benchmark,end=end)
        elif policy=='RCL':
            if verbose: 
                string = verb.print_routing_update(string,performance[0],performance[1],
                                                                                performance[2],True,benchmark,intervals=performance[3],end=end)
            RCL_averge_gap+=((performance[0]-benchmark[0])/benchmark[0])/100

print('RCL average gap:', round(RCL_averge_gap,4)*100)

********************************************************  Uchoa set Instances  ********************************************************
--------|-----------------------|	  NN 	 	|	  	  	 	RCL 	 	 	|	  GA 	 	|
Inst	|   M 	  Veh	 Obj	| t(s)	 #Veh 	 gap 	| t(s)	 #Veh 	 mean 	median	 stdev	 min	 max	| t(s)	 #Veh 	 gap 	|
--------------------------------------------------------------------------------------------------------------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Classic Instances/Uchoa/GA/X-n157-k13.pkl'

In [4]:
def create_node_range_library(instance_names):
    node_range_library = {}

    for instance_name in instance_names:
        # Extract the number of nodes from the instance name
        nodes = int(instance_name.split('-n')[1].split('-')[0])

        # Determine the range tuple
        node_range = ((nodes // 100) * 100, ((nodes // 100) * 100) + 99)

        # Add the instance to the corresponding range in the dictionary
        if node_range not in node_range_library:
            node_range_library[node_range] = [instance_name]
        else:
            node_range_library[node_range].append(instance_name)

    return node_range_library

# Example usage:
# Call the function to create the library
instance_sizes = create_node_range_library(instances['Uchoa'])

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_comparison_head(Policies,inst_set,show_gap=True)

for interval,inst_list in instance_sizes.items():

    if verbose: string = verb.print_comparison_inst(interval,len(inst_list))
    NN_gap = 0
    NN_time = 0
    RCL_gap = 0
    RCL_time = 0
    RCL_min = 0
    RCL_max = 0

    for instance in inst_list:
        purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

        with open(experiments_path+f'Uchoa/NN/{instance[:-4]}.pkl', 'rb') as file:
            NN_performance = pickle.load(file)
            NN_gap += (((NN_performance[1]-benchmark[0])/benchmark[0])/len(inst_list))
            NN_time += NN_performance[3]/len(inst_set)


        with open(experiments_path+f'Uchoa/RCL/{instance[:-4]}.pkl', 'rb') as file:
            RCL_performance = pickle.load(file)
            RCL_gap += ((RCL_performance[3][0]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_time += RCL_performance[2]/len(inst_list)/len(inst_list)
            RCL_min += ((RCL_performance[3][2]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_max += ((RCL_performance[3][3]-benchmark[0])/benchmark[0])/len(inst_list)
    

    verb.print_routing_comparison_update(string,NN_gap,NN_time,RCL_gap,RCL_time,RCL_min,RCL_max)


*****************************************  Uchoa set Instances  ******************************************
----------------|	NN 	|	     RCL 	 	|	GA 	|
Sizes	  Num 	| t(s) 	   gap 	| t(s)	  mean	  min	   max	| t(s) 	   gap 	|
---------------------------------------------------------------------------------------------
100-199	    21	| 0.01	  30.31	| 0.00	  34.19	  25.06	 105.39	|
200-299	    22	| 0.03	  27.04	| 0.00	  31.29	  22.76	 119.92	|
300-399	    15	| 0.04	  24.11	| 0.00	  29.85	  20.5	 121.53	|
400-499	    10	| 0.05	  23.9	| 0.01	  27.16	  20.04	 115.3	|
500-599	    9	| 0.07	  21.35	| 0.01	  27.32	  19.05	 118.73	|
600-699	    6	| 0.06	  27.51	| 0.03	  33.91	  25.53	 134.27	|
700-799	    6	| 0.09	  29.99	| 0.03	  36.97	  27.33	 143.59	|
800-899	    6	| 0.14	  12.91	| 0.05	  23.8	  12.65	 164.32	|
900-999	    4	| 0.12	  20.32	| 0.09	  26.77	  19.08	 102.8	|
> 1000	    1	| 0.03	  19.54	| 0.45	  45.63	  19.78	 273.3	|


In [ ]:
def create_node_range_library(instance_names):
    node_range_library = {}

    for instance_name in instance_names:
        # Extract the number of nodes from the instance name
        nodes = int(instance_name.split('-n')[1].split('-')[0])

        # Determine the range tuple
        node_range = ((nodes // 100) * 100, ((nodes // 100) * 100) + 99)

        # Add the instance to the corresponding range in the dictionary
        if node_range not in node_range_library:
            node_range_library[node_range] = [instance_name]
        else:
            node_range_library[node_range].append(instance_name)

    return node_range_library

# Example usage:
# Call the function to create the library
instance_sizes = create_node_range_library(instances['Uchoa'])

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_comparison_head(Policies,inst_set,show_gap=True)

for interval,inst_list in instance_sizes.items():

    if verbose: string = verb.print_comparison_inst(interval,len(inst_list))
    NN_gap = 0
    NN_time = 0
    RCL_gap = 0
    RCL_time = 0
    RCL_min = 0
    RCL_max = 0

    for instance in inst_list:
        purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

        with open(experiments_path+f'Uchoa/NN/{instance[:-4]}.pkl', 'rb') as file:
            NN_performance = pickle.load(file)
            NN_gap += (((NN_performance[1]-benchmark[0])/benchmark[0])/len(inst_list))
            NN_time += NN_performance[3]/len(inst_set)


        with open(experiments_path+f'Uchoa/RCL/Adaptative/{instance[:-4]}.pkl', 'rb') as file:
            RCL_performance = pickle.load(file)
            RCL_gap += ((RCL_performance[0]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_time += RCL_performance[2]/len(inst_list)/len(inst_list)
            RCL_min += ((RCL_performance[3][1]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_max += ((RCL_performance[3][2]-benchmark[0])/benchmark[0])/len(inst_list)
    

    verb.print_routing_comparison_update(string,NN_gap,NN_time,RCL_gap,RCL_time,RCL_min,RCL_max)
